## Classification

Where regression was used to predict a numeric value, classification is used to seperate data points into classes of different labels. In this example we will use a TensorFlow estimator to classify flowers.

In [40]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

### Dataset

In [41]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [42]:
# Information about each flower is the following:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
# 3 different classes of species
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

data_train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
data_test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

data_train

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0
...,...,...,...,...,...
115,5.5,2.6,4.4,1.2,1
116,5.7,3.0,4.2,1.2,1
117,4.4,2.9,1.4,0.2,0
118,4.8,3.0,1.4,0.1,0


In [43]:
y_train = data_train.pop('Species')
y_test = data_test.pop('Species')
data_train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [44]:
data_train.shape

(120, 4)

### Input Function

In [45]:
def make_input_fn(features, labels, training=True, batch_size=256):
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels)) # create tf.data.Dataset object with data and its label
    
    if training:
        dataset = dataset.shuffle(1000).repeat()  # randomize order of data
    
    return dataset.batch(batch_size)

### Feature Columns

In [46]:
feature_columns = []
for feature_name in data_train.keys():
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

### Building the Model

For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below:
- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [47]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
# n_classes = 3 ['Setosa', 'Versicolor', 'Virginica']
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[30, 10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Jhon\\AppData\\Local\\Temp\\tmplsodzvfa', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training the Model

In [48]:
# We use lambda function to call make_input_fn. As steps we use 5000 which means that we'll go through the data set until we've
# hit 5000 numbers
classifier.train(input_fn= lambda: make_input_fn(data_train, y_train), steps=5000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Jhon\AppData\Local\Temp\tmplsodzvfa\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.1870985, step = 0
INFO:tensorflow:global_step/sec: 446.645
INFO:tensorflow:loss = 1.1484133, step = 100 (0.225 sec)
INFO:tensorflow:global_step/sec: 523.941
INFO:tensorflow:loss = 1.0000985, step = 200 (0.191 sec)
INFO:tensorflow:global_step/sec: 552.949
INFO:tensorflow:loss = 0.84329957, step = 300 (0.182 sec)
INFO:tensorflow:global_step/sec: 554.801
INFO:tensorflo

 ### Evaluating the model

In [49]:
result = classifier.evaluate(input_fn= lambda: make_input_fn(data_test, y_test, training=False))
print('\nTest set accuracy: {:0.3f}\n'.format(result['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-10-29T14:01:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Jhon\AppData\Local\Temp\tmplsodzvfa\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20682s
INFO:tensorflow:Finished evaluation at 2022-10-29-14:01:08
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9, average_loss = 0.48598102, global_step = 5000, loss = 0.48598102
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\Jhon\AppData\Local\Temp\tmplsodzvfa\model.ckpt-5000

Test set accuracy: 0.900



### Predictions

In [68]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)


features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted")
for feature in features:
    valid = True
    while valid:
        value = input(feature + ": ")
        if not value.isdigit():
            valid = False
            
    predict[feature] = [float(value)]
    
predictions = classifier.predict(input_fn= lambda: input_fn(predict))
for prediction in predictions:
    id_class = prediction['class_ids'][0]
    probability = prediction['probabilities'][id_class]
    
    print('\nPrediction is "{}" ({:.1f}%)'.format(SPECIES[id_class], 100 * probability))

Please type numeric values as prompted
SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Jhon\AppData\Local\Temp\tmplsodzvfa\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Virginica" (65.8%)
